In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, './pyLDLE2/')

In [2]:
import numpy as np
from pyLDLE2 import util_, visualize_, datasets
from scipy import sparse as sp
from scipy.sparse import coo_matrix
from scipy import optimize
from scipy.special import erf, erfinv
from matplotlib import pyplot as plt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

from scipy.stats import chi2

from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
import graphlearning as gl

import bx_util

matplotlib.get_backend() =  module://matplotlib_inline.backend_inline


In [3]:
data, labels = gl.datasets.load('mnist', metric='raw')
data = data - data.mean(0)

In [4]:
default_opts = {
    'd': 2,
    'k_nn': 10,
    'k_tune': 10,
    'ds': False,
    'p': 0.99,
    'h': 1.0,
    's': 0.3,
    'local_pca': False,
    'newton_maxiter': 1000
}


def estimate_mu(X, opts=default_opts):
    d = opts['d']
    h = opts['h']
    ds = opts['ds']
    
    # compute nearest neighbors
    neigh_dist, neigh_ind = util_.nearest_neighbors(X, opts['k_nn'], metric='euclidean')
    neigh_dist = neigh_dist[:,1:]
    neigh_ind = neigh_ind[:,1:]
    
    # set h
    if h <= 0:
        h_cand = np.sqrt(2)*neigh_dist[:,opts['k_tune']-2]/np.sqrt(chi2.ppf(opts['p'], df=d))
        h = np.min(h_cand)
        
    print('h:', h, flush=True)
    
    # standard kernel
    K = np.exp(-neigh_dist**2/h**2)
    n = X.shape[0]
    source_ind = np.repeat(np.arange(n),neigh_ind.shape[1])
    K = coo_matrix((K.flatten(),(source_ind, neigh_ind.flatten())),shape=(n,n))
    ones_K_like = coo_matrix((np.ones(neigh_dist.shape).flatten(),(source_ind, neigh_ind.flatten())),shape=(n,n))

    # symmetrize the kernel
    K = K + K.T
    ones_K_like = ones_K_like + ones_K_like.T
    K.data /= ones_K_like.data
    
    # if doubly stochastic
    if ds:
        s = opts['s']
        K = K.tocoo()
        K, D = sinkhorn(K)
        K = K.tocsr()
        print('s:', s, flush=True)
    else:
        D = None
    
    # Compute ||mu_i||

    mu_norm = np.zeros_like(X)
    for i in tqdm(range(X.shape[0])):
        if opts['local_pca']:
            X_i_nbrs = X[neigh_ind[i,:].tolist()+[i],:]
            pca = PCA(n_components=d)
            y = pca.fit_transform(X_i_nbrs)
            X_rec = pca.inverse_transform(y)
            temp = X_rec[:-1,:] - X_rec[-1,:][None,:]
        else:
            temp = X[neigh_ind[i,:],:]-X[i,:][None,:]
            
        if ds:
            mu_norm[i] = K.getrow(i).power(s)[0,neigh_ind[i,:]].dot(temp)
        else:
            mu_norm[i] = K.getrow(i)[0,neigh_ind[i,:]].dot(temp)
    
    return K, mu_norm

In [5]:
W, mu = estimate_mu(data)

h: 1.0


  0%|          | 0/70000 [00:00<?, ?it/s]

In [6]:
"""We compare: SSL using
(1.) the canonical heat kernel
(2.) the ds kernel
(3.) canonical kernel + normal info
(4.) ds kernel + normal info """

""" 
For SSL, we solve the problem
\min_X { tr(X^T L X) = \sum_i w_{ij} ||X_i - X_j||^2 } s.t. X_i = Y_i on the labels

When the number of labels is small, we solve the related problem
\min_X { tr(X^T L X) = \sum_i w_{ij} ||X_i - X_j||^2 } s.t. X_i = Y_i, 1^T X = 0
"""

' \nFor SSL, we solve the problem\n\\min_X { tr(X^T L X) = \\sum_i w_{ij} ||X_i - X_j||^2 } s.t. X_i = Y_i on the labels\n\nWhen the number of labels is small, we solve the related problem\n\\min_X { tr(X^T L X) = \\sum_i w_{ij} ||X_i - X_j||^2 } s.t. X_i = Y_i, 1^T X = 0\n'

In [7]:
"""subsample labels """
num_train_per_class = 10
train_ind = gl.trainsets.generate(labels, rate=num_train_per_class)
train_labels = labels[train_ind]
F = gl.utils.labels_to_onehot(train_labels) # to one-hot labels
k = F.shape[1]

#Locations of unlabeled points
n = data.shape[0]
idx = np.full((n,), True, dtype=bool)
idx[train_ind] = False

# compute Laplacian
D = sp.spdiags((W * np.ones(n)), 0, n, n)
L = D - W

In [8]:
def conjgrad(A, b, x0=None, max_iter=1e5, tol=1e-5):
    """Conjugate Gradient Method to solve Ax = B """
    x = np.zeros_like(b)

    r = b - A@x
    p = r
    rsold = np.sum(r**2,axis=0)
  
    err = 1 
    i = 0
    while (err > tol) and (i < max_iter):
        i += 1
        Ap = A@p
        alpha = rsold / np.sum(p*Ap,axis=0)
        x += alpha * p
        r -= alpha * Ap
        rsnew = np.sum(r**2,axis=0)
        err = np.sqrt(np.sum(rsnew)) 
        p = r + (rsnew / rsold) * p
        rsold = rsnew

    return x

In [9]:
#Right hand side
b = -L[:,train_ind]*F
b = b[idx,:]

#Left hand side matrix
A = L[idx,:]
A = A[:,idx]

#Preconditioner
m = A.shape[0]
M = A.diagonal()
M = sp.spdiags(1/np.sqrt(M+1e-10),0,m,m).tocsr()

v = conjgrad(M*A*M, M*b)
v = M*v

In [10]:
#Add labels back into array
u = np.zeros((n,k))
u[idx,:] = v
u[train_ind,:] = F

pred_labels = u.argmax(1)
accuracy = np.mean(pred_labels == labels)
print(f'accuracy: {accuracy*100:.3f}%')

accuracy: 64.769%


In [11]:
from graphlearning import weightmatrix

def reweight_kernel(X, mu, r=10):
    #KNN info
    J,D = weightmatrix.knnsearch(X,r)
    
    #Normalize to unit norm
    norms = np.sqrt(np.sum(mu*mu,axis=1))
    mu = mu/norms[:,np.newaxis]
    
    V = X[:,np.newaxis,:] - X[J] #(x^0-x^i), nxkxd array
    
    # vector projection onto normals
    adotb = (V*mu[:,np.newaxis,:]).sum(-1)
    aprojb = adotb[:,:,np.newaxis] * (mu[:, np.newaxis])
    
    h=1.0
    weights = np.exp(-np.linalg.norm(aprojb, axis=-1)**2/h**2)
    
    #Flatten knn data and weights
    knn_ind = J.flatten()
    weights = weights.flatten()
    
    #Self indices
    self_ind = np.ones((n,r))*np.arange(n)[:,None]
    self_ind = self_ind.flatten()
    
    #Construct sparse matrix and symmetric normalization
    W = sp.coo_matrix((weights, (self_ind, knn_ind)),shape=(n,n)).tocsr()
    W = (W + W.transpose())/2;
    W.setdiag(0)

    return W

In [12]:
Wr = reweight_kernel(data, -mu)

# compute Laplacian
Dr = sp.spdiags((Wr * np.ones(n)), 0, n, n)
Lr = Dr - Wr

In [13]:
#Right hand side
b = -Lr[:,train_ind]*F
b = b[idx,:]

#Left hand side matrix
A = Lr[idx,:]
A = A[:,idx]

#Preconditioner
m = A.shape[0]
M = A.diagonal()
M = sp.spdiags(1/np.sqrt(M+1e-10),0,m,m).tocsr()

v = conjgrad(M*A*M, M*b)
v = M*v

In [14]:
#Add labels back into array
u = np.zeros((n,k))
u[idx,:] = v
u[train_ind,:] = F

pred_labels = u.argmax(1)
accuracy = np.mean(pred_labels == labels)
print(f'accuracy: {accuracy*100:.3f}%')

accuracy: 70.469%
